# Sentence dataset

In [1]:
# from sent_dataset import SentenceDataset
from vocabulary import Vocabulary
from utils import words_to_sent

In [13]:
import numpy as np
import nltk
from utils import sent_to_words, make_dir
from vocabulary import BOS_CODE, EOS_CODE, PAD_CODE
import pickle

class SentenceDataset:
    def __init__(self):
        self._data = None
    
    @staticmethod
    def _read_text(tokenized_text, path, voc, min_len, max_len):
        
        with open(path, 'r') as f:
            text = f.read()
            text = text.replace('\xa0', ' ').replace('\ufeff','')
            text = text.lower()

        for sentence in nltk.tokenize.sent_tokenize(text):
            words = sent_to_words(sentence)
            if min_len <= len(words) <= max_len:
                tokens = voc.to_tokens(words)
                tokenized_text.append(tokens)
    

    @property
    def shape(self):
        return list(self._data.shape)
    
    
    def build(self, paths, vocab, min_len, max_len):
        if type(paths) is str:
            paths = [paths]
        
        sentences = []
        
        for p in paths:
            self._read_text(sentences, p, vocab, min_len, max_len)

        def to_data(sent):
            assert min_len <= len(sent) <= max_len
            npads = max_len - len(sent)
            sent = [BOS_CODE] + sent + [EOS_CODE] + [PAD_CODE] * npads
            assert len(sent) == max_len + 2
            return np.array(sent, dtype=np.int32)
            
        self._data = np.zeros([len(sentences), max_len+2], dtype=np.int32)
        for i in range(len(sentences)):
            self._data[i] = to_data(sentences[i])
            
            
    def save(self, path):
        make_dir(path)
        pickle.dump([self._data], open(path, "wb"))


    def restore(self, path):
        [self._data] = pickle.load(open(path, "rb"))

    
    def get_next_batch(self, bs):
        num   = len(self._data)
        idx   = np.random.choice(np.arange(num), bs, replace=bs>num)
        sents = self._data[idx]
        return sents


## Vocabulary

In [14]:
voc_path = "vocabulary.data"
voc = Vocabulary()
voc.restore(voc_path)
print(voc.size)

100004


## Build

In [15]:
%%time

dataset = SentenceDataset()
dataset.build(
    paths   = 'data/anna.txt', # 'data/dostoevsky.txt'
    vocab   = voc,
    min_len = 2, 
    max_len = 20
)
print(dataset.shape)

assert dataset.shape[1] == 22

[14117, 22]
CPU times: user 4.48 s, sys: 26.7 ms, total: 4.51 s
Wall time: 4.5 s


## Save/restore

In [12]:
dataset_path = "./dataset.data"
dataset.save(dataset_path)
old = dataset
dataset = SentenceDataset()
dataset.restore(dataset_path)
print(dataset.shape)

AttributeError: 'SentenceDataset' object has no attribute '_tokens_to_words'

## Batch

In [54]:
batch = dataset.get_next_batch(15)
for sent in batch:
    restored = words_to_sent(voc.to_words(sent))
    restored = restored.replace(' <PAD>', '_')
    print(restored)

<BOS> — велела она мне, оставляя меня с альфонсинкой, — и там умри, если надо, понимаешь? <EOS>
<BOS> он двинулся было с места, но четверо, его <UNK>, вдруг разом схватили его за руки. <EOS>_
<BOS> зачем мне тогда приснилось дитё в такую минуту? <EOS>___________
<BOS> ненависть загорелась в моем сердце. <EOS>______________
<BOS> кроме вас, никто другой не поймет, а он во главе всех других. <EOS>_____
<BOS> — в статье всего этого нет, там только намеки, — проговорил раскольников. <EOS>_____
<BOS> — что ж, мне так и оставаться голым? <EOS>__________
<BOS> он здесь живет; бывший крепостной человек, ну, вот пощечину дал. <EOS>______
<BOS> — тут хорошо, — ободряла бабенка. <EOS>____________
<BOS> — никак невозможно-с. <EOS>________________
<BOS> алеша помнил потом, что она показалась ему чрезвычайно хороша собой в ту минуту. <EOS>_____
<BOS> адвокат в этом случае правду сказал. <EOS>_____________
<BOS> как она вздрогнула! <EOS>________________
<BOS> я черных тараканов ночью туфлей <UNK>: та